In [1]:
import csv
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
base = "CME"

threshold = 3.5

file_CME = "Input Data/detections_CME.csv"
file_Dino = "Input Data/detections_Dino.csv"

In [3]:
df_CME = pd.read_csv(file_CME, header = None)
M_CME = df_CME.to_numpy()
M_CME[:, 4] = M_CME[:, 4]-1

df_Dino = pd.read_csv(file_Dino)
df_Dino = df_Dino[["timepoint","label_id","x","y","z","intensity","max_intensity"]]
df_Dino = df_Dino.drop("max_intensity", axis=1)
M_Dino = df_Dino.to_numpy()
M_Dino[:, 0] += 1

if base == "CME":
    comp_list = np.zeros((len(M_CME),15), dtype=object)
    M_x = M_CME
    M_y = M_Dino
    sec = "Dino"
if base == "Dino":
    comp_list = np.zeros((len(M_Dino),15), dtype=object)
    M_x = M_Dino
    M_y = M_CME
    sec = "CME"

In [4]:
t_vec = defaultdict(list)
multi_match_list = []
for vec_y in M_y:
    t_vec[vec_y[0]].append(vec_y)

for t_val in t_vec:
    t_vec[t_val] = np.array(t_vec[t_val])

for i, vec_x in enumerate(M_x):
    t_val = vec_x[0]
    y_group = t_vec[t_val]
    diffs = y_group[:, 2:5] - vec_x[2:5]
    dists = np.linalg.norm(diffs, axis=1)
    min_idx = np.argmin(dists)
    min_dist = dists[min_idx]
    best_vec_y = y_group[min_idx]
    comp_list[i, 0:6] = vec_x
    comp_list[i, 6:12] = best_vec_y
    comp_list[i, 12] = min_dist

    below = np.where(dists < threshold)[0]
    if len(below) > 0:
        y_ids = ",".join(str(int(x)) for x in y_group[below, 1])
        dists_str = ",".join(f"{dists[j]:.2f}" for j in below)
    else:
        y_ids = ""
        dists_str = ""
    comp_list[i, 13] = y_ids
    comp_list[i, 14] = dists_str

multi_match_array = np.array(multi_match_list, dtype=object)

In [5]:
df_comp = pd.DataFrame(comp_list, columns = ["t",f"ID ({base})", f"x ({base})", f"y ({base})", 
                                             f"z ({base})", f"FI ({base})", "t_ig", f"ID ({sec})", 
                                             f"x ({sec})", f"y ({sec})", f"z ({sec})", 
                                             f"FI ({sec})", "Distance", f"Multi ID ({sec})", f"Multi Distance ({sec})"])

df_comp = df_comp.drop("t_ig", axis=1)
df_sorted = df_comp.sort_values(by=[f"ID ({base})","t"], ascending=[True, True])

In [6]:
df_sorted.to_csv(f"{base} Base Output/DetectionComparison.csv", index=False)